<a href="https://colab.research.google.com/github/linuxsynckliye/ELL784_assignment2/blob/master/fisherFaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import cv2
import os
import numpy as np

# number of eigen vectors to retain
n_eig_components = 6

# =========================================================================================
# store images as vector for each class
# =========================================================================================

class Subject:
    def __init__(self, data):
        self.data_subject = data

    def appendData(self,data):
        self.data_subject = np.column_stack((data,self.data_subject))


# =========================================================================================
# finding W_pca
# =========================================================================================

def pca(X, n_components):

  mU_overall = np.mat(X.mean(axis = 1)).T

  X = X - mU_overall
  # to find eigen vectors of XX' we first calculate eigen vectors of X'X
  # then pre-multiply it with matrix X
  XTX = np.dot(X.T, X)

  [e_val_PCA, e_vec_PCA] = np.linalg.eig(XTX)
  e_vec_PCA = np.dot(X, e_vec_PCA)

  for i in range(e_vec_PCA.shape[1]):
    e_vec_PCA[:,i] = e_vec_PCA[:,i] / np.linalg.norm(e_vec_PCA[:,i])

  # sort according to decreasing value of eigen_values
  index_sorted = np.argsort(-1*(e_val_PCA))

  e_val_PCA = e_val_PCA[index_sorted]
  e_vec_PCA = e_vec_PCA[:, index_sorted]

  print("size of eigenVector matrix for PCA is :", e_vec_PCA.shape)

  # return first 'n_components' eigen vectors/ values and overall mean
  return([ e_val_PCA[0 : n_components], e_vec_PCA[:, 0 : n_components], mU_overall ])
  

# =========================================================================================
# finding W_fld
# =========================================================================================

def fld(Xprojected, mU_overall, n_classes, n_components, e_vec_pca):

  [row_x, col_x] = Xprojected.shape
  mU_overall_p = np.mat(Xprojected.mean(axis = 1))

  # obtaining scatter matrices
  Sw = np.zeros((row_x, row_x), dtype = np.float32)
  Sb = np.zeros((row_x, row_x), dtype = np.float32)


  for i in range(n_classes):

    Xi = np.dot(e_vec_pca.T, subjects[i].data_subject - mU_overall)
    
    # mean of each class
    mU_class = np.mat(Xi.mean(axis = 1))

    # within class scatter matrix
    Sw += np.dot((Xi - mU_class), (Xi - mU_class).T)
    N_i = Xi.shape[1]

    # between class scatter matrix
    Sb += N_i * np.dot((mU_overall_p - mU_class) , (mU_overall_p - mU_class).T)

  [e_val_FLD, e_vec_FLD] = np.linalg.eig(np.dot(np.linalg.inv(Sw),Sb))

  index_sorted = np.argsort(-1*(e_val_FLD.real))
  
  e_val_FLD = e_val_FLD[index_sorted]
  e_vec_FLD = e_vec_FLD[:, index_sorted]

  print("size of eigenVector matrix for FLD is :", e_vec_FLD.shape)

  eigenvalues = np.array ( e_val_FLD [0: n_components ].real , dtype = np.float32 , copy = True )
  eigenvectors = np.array ( e_vec_FLD [0: ,0: n_components ].real , dtype = np.float32 , copy = True )
  
  print("size of eigenVector matrix (returned) for FLD is :", e_vec_FLD.shape)

  return ([ eigenvalues, eigenvectors])

# =========================================================================================
# read all files in folder
# image class is assumed to be mentioned as file name itself
# =========================================================================================

files = os.listdir('/content/drive/My Drive/Photos/yalefacesReducedDataset')

img = cv2.imread('/content/drive/My Drive/Photos/yalefacesReducedDataset/'+files[0],cv2.IMREAD_GRAYSCALE)
row, col = img.shape
data = np.reshape(img,(row*col,1))

print("###shape###", row,col)

# this keeps track of which class number corresponds to which subject
dict = {}

# final value of key_dict serves as number of classes in training set also
key_dict = 0

subjects = []

for file in files:
    # print(".....................................")
    
    img = cv2.imread('/content/drive/My Drive/Photos/yalefacesReducedDataset/'+file,cv2.IMREAD_GRAYSCALE)

    data = np.reshape(img,(row*col,1))

    if file[0:9] not in dict:
        dict[file[0:9]] = key_dict

        subjects.append(Subject(data))
        key_dict += 1

    else:
        subject_index = dict[file[0:9]]
        subjects[subject_index].appendData(data)

X = np.empty([row*col, 1])
X = np.delete(X,0,1)

# generate matrix of all images as column vectors
for i in range(key_dict):
    X = np.column_stack((subjects[i].data_subject,X))


[e_val_pca, e_vec_pca, mU_overall] = pca(X, n_eig_components)

[e_val_fld, e_vec_fld] = fld(np.dot( e_vec_pca.T,(X - mU_overall)), mU_overall, key_dict, n_eig_components, e_vec_pca)

# obtain transformation matrix (W) that projects images into C-1 dimensional space
# =========================================================================================
# W = W_pca * W_fld
# =========================================================================================

W = np.dot(e_vec_pca, e_vec_fld)
print("size of transformation matrix W is :", W.shape)


test = cv2.imread('/content/drive/My Drive/Photos/subject03_09.png', cv2.IMREAD_GRAYSCALE)
# reshape the image as column vector
test_v = np.reshape(test,(row*col,1))
print("size of test_image vector is :", test_v.shape)

# take projection 
testProjection = np.dot(W.T, test_v-mU_overall)
print("size of projection of test_image vector is :", testProjection.shape)

result_class = -1
minDistance = 0

# W = e_vec_pca
for i in range(key_dict):
  for j in range(subjects[i].data_subject.shape[1]):
    sub_Ci_Sj = np.mat(subjects[i].data_subject[:,j]).T

    subProj = np.dot(W.T, sub_Ci_Sj - mU_overall)
    
    distance = np.linalg.norm(testProjection - subProj)
    # print("aaya bhi ya nai        ",distance)

    # initialize distance for first iteration
    if(i==0 and j==0):
      minDistance = distance
      result_class = i

      continue

    if distance < minDistance:
      print("updating result class to ", i)
      result_class = i
      minDistance = distance

# print("raat bhar ka mehnat",result_class)

# =========================================================================================
# Display results
# =========================================================================================
print("test subject belongs to : ",list(dict.keys())[list(dict.values()).index(result_class)])
# print(dict)


###shape### 243 320
size of eigenVector matrix for PCA is : (77760, 30)
size of eigenVector matrix for FLD is : (6, 6)
size of eigenVector matrix (returned) for FLD is : (6, 6)
size of transformation matrix W is : (77760, 6)
size of test_image vector is : (77760, 1)
size of projection of test_image vector is : (6, 1)
updating result class to  2
updating result class to  2
updating result class to  2
test subject belongs to :  subject03
